In [71]:
import pandas as pd
from Levenshtein import distance
import jieba

In [72]:
jieba.load_userdict('../data/dict.txt')
df = pd.read_csv('../data/all.csv',encoding='utf-8-sig')
df.head()

,s1,s2,label
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
2,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
3,如何得知关闭借呗,想永久关闭借呗,0
4,花呗扫码付钱,二维码扫描可以用花呗吗,0


In [73]:
a = jieba.lcut(df['s1'][0])
print(a)
a.remove('\ufeff')
print(a)

['\ufeff', '怎么', '更改', '花呗', '手机号码']
['怎么', '更改', '花呗', '手机号码']


In [74]:
df['len_s1'] = df.apply(lambda x: len(x['s1']),axis=1)
df['len_s2'] = df.apply(lambda x: len(x['s2']),axis=1)
df['len_s1s2_avg'] = (df['len_s1']+df['len_s2'])/2
df['len_s1_on_s2'] = df['len_s1']/df['len_s2']

df['len_diff'] = df.apply(lambda x: len(x['s1'])-len(x['s2']),axis=1)
df['edit_dst'] = df.apply(lambda x: distance(x['s1'],x['s2']),axis=1)

df['edit_dst_nzd'] = df['len_s1s2_avg']/df['edit_dst']

In [75]:
def get_counts(row):
    s1, s2 = row['s1'], row['s2']
    list_s1 = jieba.lcut(s1)
    list_s2 = jieba.lcut(s2)
    to_remove = ['， ','，','*','* ','!','! ','？','？ ','。','。 ','','\ufeff']
    list_s1_cnd = [x for x in list_s1 if x not in to_remove]
    list_s2_cnd = [x for x in list_s2 if x not in to_remove]
    set_s1 = set(list_s1_cnd)
    set_s2 = set(list_s2)
    return list_s1_cnd, list_s2_cnd, \
           len(list_s1_cnd), len(list_s2_cnd), \
           len(set_s1.intersection(set_s2)), \
           len(set_s1-set_s2), len(set_s2-set_s1) \


df[['l1_tkd','l2_tkd','len_l1_tkd','len_l2_tkd','common_wc','s1_uq_wc','s2_uq_wc']]=df.apply(get_counts, axis=1,result_type='expand')

In [92]:
df['l1_tkd'][0]

['怎么', '更改', '花呗', '手机号码']

In [77]:
def find_askwords(row):
    s1, s2 = row['l1_tkd'], row['l2_tkd']
    ask_words = ['如何','怎么','为什么','为何','咋么']
    s1_ask, s2_ask = False, False
    if len(s1)>0 and s1[0] in ask_words:
        s1_ask = True
    if len(s2)>0 and s2[0] in ask_words:
        s2_ask = True
    return s1_ask, s2_ask

df[['s1_ask','s2_ask']] = df.apply(find_askwords, axis=1,result_type='expand')
df.head()


,s1,s2,label,len_s1,len_s2,len_s1s2_avg,len_s1_on_s2,len_diff,edit_dst,edit_dst_nzd,l1_tkd,l2_tkd,len_l1_tkd,len_l2_tkd,common_wc,s1_uq_wc,s2_uq_wc,s1_ask,s2_ask
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,11,30,20.5,0.366667,-19,24,0.854167,"[怎么, 更改, 花呗, 手机号码]","[我, 的, 花呗, 是, 以前, 的, 手机号码, 怎么, 更, 改成, 现在, 的, 支...",4,16,3,1,11,True,False
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,15,10,12.5,1.500000,5,14,0.892857,"[也, 开, 不了, 花呗, 就, 这样, 了, 完事, 了]","[真的, 嘛, 就是, 花呗, 付款]",9,5,1,7,5,False,False
2,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,11,13,12.0,0.846154,-2,11,1.090909,"[花呗, 冻结, 以后, 还, 能, 开通, 吗]","[我, 的, 条件, 可以, 开通, 花呗, 借款, 吗]",7,8,3,4,5,False,False
3,如何得知关闭借呗,想永久关闭借呗,0,8,7,7.5,1.142857,1,4,1.875000,"[如何, 得知, 关闭, 借呗]","[想, 永久, 关闭, 借呗]",4,4,2,2,2,True,False
4,花呗扫码付钱,二维码扫描可以用花呗吗,0,6,11,8.5,0.545455,-5,10,0.850000,"[花呗, 扫码, 付钱]","[二维码, 扫描, 可以, 用, 花呗, 吗]",3,6,1,2,5,False,False


In [78]:
df['len_l1l2_tkd_avg'] = (df['len_l1_tkd'] + df['len_l2_tkd']) / 2
df['common_wc_nzd'] = df['common_wc'] / df['len_l1l2_tkd_avg']
df['s1_uq_wc_nzd'] = df['s1_uq_wc']/df['len_l1_tkd']
df['s2_uq_wc_nzd'] = df['s2_uq_wc']/df['len_l2_tkd']


df.head()

,s1,s2,label,len_s1,len_s2,len_s1s2_avg,len_s1_on_s2,len_diff,edit_dst,edit_dst_nzd,...,len_l2_tkd,common_wc,s1_uq_wc,s2_uq_wc,s1_ask,s2_ask,len_l1l2_tkd_avg,common_wc_nzd,s1_uq_wc_nzd,s2_uq_wc_nzd
0,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1,11,30,20.5,0.366667,-19,24,0.854167,...,16,3,1,11,True,False,10.0,0.300000,0.250000,0.687500
1,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0,15,10,12.5,1.500000,5,14,0.892857,...,5,1,7,5,False,False,7.0,0.142857,0.777778,1.000000
2,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0,11,13,12.0,0.846154,-2,11,1.090909,...,8,3,4,5,False,False,7.5,0.400000,0.571429,0.625000
3,如何得知关闭借呗,想永久关闭借呗,0,8,7,7.5,1.142857,1,4,1.875000,...,4,2,2,2,True,False,4.0,0.500000,0.500000,0.500000
4,花呗扫码付钱,二维码扫描可以用花呗吗,0,6,11,8.5,0.545455,-5,10,0.850000,...,6,1,2,5,False,False,4.5,0.222222,0.666667,0.833333


In [79]:
from nltk.translate.bleu_score import sentence_bleu

def get_ngram_bleu(row):
    s1, s2 = row['s1'], row['s2']
    list_s1 = [[x for x in s1]]
    list_s2 = [x for x in s2]
    b1 = sentence_bleu(list_s1, list_s2, weights=(1, 0, 0, 0))
    b2 = sentence_bleu(list_s1, list_s2, weights=(0.5, 0.5, 0, 0))
    b3 = sentence_bleu(list_s1, list_s2, weights=(0.33, 0.33, 0.33, 0))
    b4 = sentence_bleu(list_s1, list_s2, weights=(0.25, 0.25, 0.25, 0.25))
    return b1, b2, b3, b4

df[['bleu1','bleu2','bleu3','bleu4']]=df.apply(get_ngram_bleu, axis=1,result_type='expand')

/Users/FY/opt/anaconda3/envs/py36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/FY/opt/anaconda3/envs/py36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/FY/opt/anaconda3/envs/py36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider usi

In [91]:
df.iloc[0]

s1                                                        ﻿怎么更改花呗手机号码
s2                                     我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号
label                                                               1
len_s1                                                             11
len_s2                                                             30
len_s1s2_avg                                                     20.5
len_s1_on_s2                                                 0.366667
len_diff                                                          -19
edit_dst                                                           24
edit_dst_nzd                                                 0.854167
l1_tkd                                             [怎么, 更改, 花呗, 手机号码]
l2_tkd              [我, 的, 花呗, 是, 以前, 的, 手机号码, 怎么, 更, 改成, 现在, 的, 支...
len_l1_tkd                                                          4
len_l2_tkd                                                         16
common_wc           

In [ ]:
import pandas as pd
from Levenshtein import distance
import jieba

In [ ]:
jieba.load_userdict('../data/dict.txt')
df = pd.read_csv('../data/all.csv',encoding='utf-8-sig')
df.head()

In [ ]:
a = jieba.lcut(df['s1'][0])
print(a)
a.remove('\ufeff')
print(a)

In [ ]:
df['len_s1'] = df.apply(lambda x: len(x['s1']),axis=1)
df['len_s2'] = df.apply(lambda x: len(x['s2']),axis=1)
df['len_s1s2_avg'] = (df['len_s1']+df['len_s2'])/2
df['len_s1_on_s2'] = df['len_s1']/df['len_s2']

df['len_diff'] = df.apply(lambda x: len(x['s1'])-len(x['s2']),axis=1)
df['edit_dst'] = df.apply(lambda x: distance(x['s1'],x['s2']),axis=1)

df['edit_dst_nzd'] = df['len_s1s2_avg']/df['edit_dst']

In [ ]:
def get_counts(row):
    s1, s2 = row['s1'], row['s2']
    list_s1 = jieba.lcut(s1)
    list_s2 = jieba.lcut(s2)
    to_remove = ['， ','，','*','* ','!','! ','？','？ ','。','。 ','','\ufeff']
    list_s1_cnd = [x for x in list_s1 if x not in to_remove]
    list_s2_cnd = [x for x in list_s2 if x not in to_remove]
    set_s1 = set(list_s1_cnd)
    set_s2 = set(list_s2)
    return list_s1_cnd, list_s2_cnd, \
           len(list_s1_cnd), len(list_s2_cnd), \
           len(set_s1.intersection(set_s2)), \
           len(set_s1-set_s2), len(set_s2-set_s1) \


df[['l1_tkd','l2_tkd','len_l1_tkd','len_l2_tkd','common_wc','s1_uq_wc','s2_uq_wc']]=df.apply(get_counts, axis=1,result_type='expand')

In [ ]:
df['l1_tkd'][0]

In [ ]:
def find_askwords(row):
    s1, s2 = row['l1_tkd'], row['l2_tkd']
    ask_words = ['如何','怎么','为什么','为何','咋么']
    s1_ask, s2_ask = False, False
    if len(s1)>0 and s1[0] in ask_words:
        s1_ask = True
    if len(s2)>0 and s2[0] in ask_words:
        s2_ask = True
    return s1_ask, s2_ask

df[['s1_ask','s2_ask']] = df.apply(find_askwords, axis=1,result_type='expand')
df.head()


In [ ]:
df['len_l1l2_tkd_avg'] = (df['len_l1_tkd'] + df['len_l2_tkd']) / 2
df['common_wc_nzd'] = df['common_wc'] / df['len_l1l2_tkd_avg']
df['s1_uq_wc_nzd'] = df['s1_uq_wc']/df['len_l1_tkd']
df['s2_uq_wc_nzd'] = df['s2_uq_wc']/df['len_l2_tkd']


df.head()

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def get_ngram_bleu(row):
    s1, s2 = row['s1'], row['s2']
    list_s1 = [[x for x in s1]]
    list_s2 = [x for x in s2]
    b1 = sentence_bleu(list_s1, list_s2, weights=(1, 0, 0, 0))
    b2 = sentence_bleu(list_s1, list_s2, weights=(0.5, 0.5, 0, 0))
    b3 = sentence_bleu(list_s1, list_s2, weights=(0.33, 0.33, 0.33, 0))
    b4 = sentence_bleu(list_s1, list_s2, weights=(0.25, 0.25, 0.25, 0.25))
    return b1, b2, b3, b4

df[['bleu1','bleu2','bleu3','bleu4']]=df.apply(get_ngram_bleu, axis=1,result_type='expand')

In [ ]:
df.iloc[0]

In [ ]:
df.to_csv('../data/trd_feated.csv',index=False)

In [ ]:
df.shape

In [88]:
df.to_csv('../data/trd_feated.csv',index=False)

In [89]:
df.shape

(102477, 27)